In [1]:
import pandas as pd
#df = pd.read_csv('datasets/erroneuous_test_data_set.csv')
df = pd.read_csv('datasets/adult.csv')


In [6]:
df2 = df[:10]
print(df2)
df2.to_excel('datasets/synthetic_data_adult.xlsx')

   age         workclass  fnlwgt     education  educational-num  \
0   25           Private  226802          11th                7   
1   38           Private   89814       HS-grad                9   
2   28         Local-gov  336951    Assoc-acdm               12   
3   44           Private  160323  Some-college               10   
4   18                 ?  103497  Some-college               10   
5   34           Private  198693          10th                6   
6   29                 ?  227026       HS-grad                9   
7   63  Self-emp-not-inc  104626   Prof-school               15   
8   24           Private  369667  Some-college               10   
9   55           Private  104996       7th-8th                4   

       marital-status         occupation   relationship   race  gender  \
0       Never-married  Machine-op-inspct      Own-child  Black    Male   
1  Married-civ-spouse    Farming-fishing        Husband  White    Male   
2  Married-civ-spouse    Protective-serv

In [77]:
import pandas as pd
import numpy as np
import random
# suppose your existing dataframe is df
dataset = pd.read_csv('datasets/adult.csv') # replace with your file
df = dataset[:50].copy() # create a copy of the first 50 rows
df.drop(['educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'hours-per-week','capital-gain', 'capital-loss'], axis=1, inplace=True)


#generate missingness in 10% of data
amount_missing = int(0.1* len(df))
missing_positions = [(row, col) for row in range(df.shape[0]) for col in range(df.shape[1])]
for row, col in random.sample(missing_positions, amount_missing):
    df.iat[row, col] = np.nan if random.random() > 0.5 else None

# Select a fraction of the 'age' column to be outliers
amount_outliers = int(0.1 * len(df))
# Select the indices of the rows to be made into outliers
outlier_indices = np.random.choice(df.index, amount_outliers, replace=False)
print(outlier_indices)
# Add a large number (e.g., 100) to the ages of these rows to create outliers
df.loc[outlier_indices, 'age'] = df.loc[outlier_indices, 'age'] + 100


#generate random variations in categorical string oclumns
for col in df.select_dtypes(include=[object]).columns:
    df[col] = df[col].apply(lambda x: x if (random.random() > 0.05 or pd.isnull(x)) else x + random.choice(['VAR1', 'var2', 'V@R3', '4','five',]))

# Creating a column with mixed data types
df['mixed_data_types'] = pd.concat([pd.Series(np.random.choice([1, 2, 3], df.shape[0]//2)), pd.Series(np.random.choice(['MD1', 'MD2', 'MD3'], df.shape[0] - df.shape[0]//2))]).sample(frac=1).reset_index(drop=True)

# Adding special characters
random_col = random.choice(df.columns)
special_char_col = df[random_col].sample(frac=0.1, replace=True) # 10% of rows
special_char_col = special_char_col.apply(lambda x: random.choice(['@#$@', '###']) if isinstance(x, str) else x)
df.update(special_char_col)

# Creating conflicting labels
conflict_df = df.sample(frac=0.1, replace=True) # 10% of rows
conflict_df['income'] = conflict_df['income'].apply(lambda x: '<=50K' if x=='>50K' else '>50K')
df.update(conflict_df)

# Creating class imbalance
df['income'] = df['income'].apply(lambda x: x if random.random() > 0.8 else ('<=50K' if x=='>50K' else '>50K')) # 80-20 imbalance


#replace some rows for duplicates
to_replace = df.sample(frac=0.1).index
duplicate_rows_df = df.sample(n=len(to_replace), replace=True)
duplicate_rows_df.index = to_replace

# Replace the rows
df.loc[to_replace] = duplicate_rows_df

# Duplicate the column
random_col = random.choice(df.columns)
df['duplicate column'] = df[random_col]

# Creating a column with a single unique value (redundant column)
df['redundant_SV_column'] = 'this_is_the_same'


df = df.reset_index(drop=True)



[27 33 42 47 24]


In [78]:
print(df)
print(df.columns)
print(len(df))
df.to_excel('datasets/synthetic_test_dataset.xlsx', index=False)

      age             workclass  fnlwgt      education      gender  \
0    25.0               Private  226802           11th        Male   
1    37.0               Private   60548        HS-grad      Female   
2    28.0             Local-gov  336951     Assoc-acdm        Male   
3    44.0               Private  160323   Some-college        Male   
4    18.0                     ?  103497   Some-college      Female   
5    34.0               Private  198693           10th    MaleVAR1   
6    29.0                     ?  227026        HS-grad        Male   
7    63.0      Self-emp-not-inc  104626    Prof-school    Malefive   
8    24.0               Private  369667   Some-college      Female   
9    55.0               Private  104996        7th-8th    Malevar2   
10   65.0               Private  184454        HS-grad        Male   
11   36.0           Federal-gov  212465      Bachelors        Male   
12   26.0               Private   82091        HS-grad      Female   
13   58.0           

In [57]:
import pandas as pd
import numpy as np
import random
# suppose your existing dataframe is df
dataset = pd.read_csv('datasets/adult.csv') # replace with your file
df = dataset[:50].copy() # create a copy of the first 50 rows
df_copy = df.copy() # create another copy to drop columns from
df_copy.drop(['educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'hours-per-week','capital-gain', 'capital-loss'], axis=1, inplace=True)
print(df)

    age         workclass  fnlwgt     education  educational-num  \
0    25           Private  226802          11th                7   
1    38           Private   89814       HS-grad                9   
2    28         Local-gov  336951    Assoc-acdm               12   
3    44           Private  160323  Some-college               10   
4    18                 ?  103497  Some-college               10   
5    34           Private  198693          10th                6   
6    29                 ?  227026       HS-grad                9   
7    63  Self-emp-not-inc  104626   Prof-school               15   
8    24           Private  369667  Some-college               10   
9    55           Private  104996       7th-8th                4   
10   65           Private  184454       HS-grad                9   
11   36       Federal-gov  212465     Bachelors               13   
12   26           Private   82091       HS-grad                9   
13   58                 ?  299831       HS-grad 